# Data Slicing Performance

Exercise repository: [Performance_testing_FinalExercise](https://github.com/mxagar/mlops-udacity-deployment-demos/tree/main/Performance_testing_FinalExercise).

In this exercise the [Raisin dataset](https://archive.ics.uci.edu/ml/datasets/Raisin+Dataset) is used. In it, 900 observations of 7 raising features (all numerical) are collected; each observation has a target class which specifies one of two types of raisins: 'Kecimen' or 'Besni'.

Data slicing is applied manually to the dataset as follows:

- Data is split: `train`, `validation`.
- Logistic regression model is fit and F1 is computed: overall score.
- 3 features are chosen and their mean is computed; then, 2 buckets or ranges are defined for each of the 3 features: above and below the mean. Thus, we get 3x2 = 6 slices.
- For each slice, the F1 metric is computed again and compared to the overall F1.
- Finally, a model card is written.

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, f1_score

In [14]:
data = pd.read_csv("./exercise_data/Raisin_Dataset.csv")

In [15]:
data.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,Kecimen
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,Kecimen
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,Kecimen
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,Kecimen
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,Kecimen


In [16]:
data.head()

(900, 8)

In [17]:
data.Class.unique()

array(['Kecimen', 'Besni'], dtype=object)

In [11]:
# Take features + target
y = data.pop("Class")

# Split the data into train and validation, stratifying on the target feature.
X_train, X_val, y_train, y_val = train_test_split(data, y, stratify=y, random_state=23)

In [3]:
# Get a high level overview of the data.
# This will be useful for slicing: we take the mean values from here!
X_train.describe()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter
count,675.000000,675.000000,675.000000,675.000000,675.000000,675.000000,675.000000
mean,87210.494815,427.650555,254.414345,0.779895,90407.262222,0.701092,1159.625772
std,38388.571707,110.506268,49.752074,0.088938,39602.352484,0.050807,261.820857
min,25387.000000,225.629541,144.618672,0.348730,26139.000000,0.454189,619.074000
25%,59032.500000,343.732369,218.692197,0.740516,61466.500000,0.671134,964.835500
50%,79057.000000,405.936594,247.352044,0.797864,81779.000000,0.709949,1117.107000
75%,103790.500000,493.185891,280.180509,0.840452,108022.500000,0.735886,1302.416500
max,235047.000000,843.956653,492.275279,0.923770,239093.000000,0.830632,2253.557000


In [4]:
# Define and fit model
lr = LogisticRegression(max_iter=1000, random_state=23)
lb = LabelBinarizer()

# Binarize the target feature.
y_train = lb.fit_transform(y_train)
y_val = lb.transform(y_val)

# Train Logistic Regression.
lr.fit(X_train, y_train.ravel())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=23, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# Solution Code

In [5]:
# Use sklearn's classification report to get an overall view of our classifier.
print(classification_report(y_val, lr.predict(X_val)))

              precision    recall  f1-score   support

           0       0.90      0.86      0.88       112
           1       0.86      0.90      0.88       113

    accuracy                           0.88       225
   macro avg       0.88      0.88      0.88       225
weighted avg       0.88      0.88      0.88       225



In [6]:
print("F1 score on MajorAxisLength slices:")
row_slice = X_val["MajorAxisLength"] >= 427.7
print(f1_score(y_val[row_slice], lr.predict(X_val[row_slice])))

row_slice = X_val["MajorAxisLength"] < 427.7
print(f1_score(y_val[row_slice], lr.predict(X_val[row_slice])))

print("\nF1 score on MinorAxisLength slices:")
row_slice = X_val["MinorAxisLength"] >= 254.4
print(f1_score(y_val[row_slice], lr.predict(X_val[row_slice])))

row_slice = X_val["MinorAxisLength"] < 254.4
print(f1_score(y_val[row_slice], lr.predict(X_val[row_slice])))

print("\nF1 score on ConvexArea slices:")
row_slice = X_val["ConvexArea"] >= 90407.3
print(f1_score(y_val[row_slice], lr.predict(X_val[row_slice])))

row_slice = X_val["ConvexArea"] < 90407.3
print(f1_score(y_val[row_slice], lr.predict(X_val[row_slice])))

F1 score on MajorAxisLength slices:
0.0
0.9230769230769231

F1 score on MinorAxisLength slices:
0.65
0.9319371727748692

F1 score on ConvexArea slices:
0.30769230769230765
0.9174311926605505


From the above slices, we see that our model seems to perform consistently better on raisins that are smaller than average versus ones that are larger than average.

Looking at the summary statistics, we see that for nearly every measure the median is smaller than the mean. So more than 50% of our raisins are below the average which is also where our model is strongest. This indicates that we might want more data on larger raisins.

## Model Card

### Model Details

Logistic Regresion model using default scikit-learn hyperparameters. Trained with sklearn version 0.24.1.

### Intended Use

For classifying two types of raisins from Turkey.

### Metrics

F1 classification with a macro average of 0.85, 0.84 for the minority class, and 0.85 for the majority class.

When analyzing across data slices, model performance is higher for raisins below the average size and much lower for raisins above the average.

### Data

Raisin dataset acquired from the UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/Raisin+Dataset

Originally from: Cinar I., Koklu M. and Tasdemir S., Classification of Raisin Grains Using Machine Vision and Artificial Intelligence Methods. Gazi Journal of Engineering Sciences, vol. 6, no. 3, pp. 200-209, December, 2020.

### Bias

The majority of raisins are below the average size. This could be a potential source of bias but more subject matter expertise may be necessary. Note to students: this is a useful call out, and in a real-world scenario should prompt you to engage in collaboration with subject matter experts so you can flesh this out.